In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score 
import time

## Открытие и изучение данных

Загрузим данные и посмотрим общую информацию:

In [2]:
try:

    df = pd.read_csv('users_behavior.csv', sep=',')
    
except:
    
    df = pd.read_csv('/datasets/users_behavior.csv', sep=',')
    
pd.options.display.max_columns = None
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Первые пять строк данных:

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Последние пять строк данных:

In [4]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


Проверка на дубликаты:

In [5]:
df.duplicated().sum()

0

Дубликатов нет.

### Выводы:

Описание данных:
*	сalls — количество звонков,
*	minutes — суммарная длительность звонков в минутах,
*	messages — количество sms-сообщений,
*	mb_used — израсходованный интернет-трафик в Мб,
*	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

В данных нет пропусков и дубликатов они готовы для последующей работы.

Так как нужно на основе этих данных обучить модель которая будет рекомендовать тариф
то target это данные столбца is_ultra, а features все остальные данные за исключением is_ultra.



## Разделение данных на обучающую, валидационную и тестовую выборки

Подготовим данные к класификации сохраним признаки в переменной features, а целевой признак в переменной target: 

In [6]:
features = df.drop('is_ultra',axis=1)
target = df['is_ultra']

Так как нужно создать сначала тестовую выборку применим метод train_test_split дважды:
* Разделим данные на 0.8 и 0.2
* Разделим получившиеся данные на 0.25 и 0.75 
В результате получим правильное разделение данных на обучающие 0.6 и валидационные 0.2 и тестовые 0.2

Получим тестовую выборку:

In [7]:
features_80, features_test, target_80, target_test = train_test_split(
    features, target, test_size=0.2, random_state=1879)

Получим валидационную выборку и обучающие данные:

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_80, target_80, test_size=0.25, random_state=1879)

Теперь есть три набора данных обучающая выборка, валидационная и тестовая.

## Исследование качества разных моделей

Для того что бы понять как обучить модель нужно проверить какя модель лучше справится с нашими данными, нужно посмотреть на качества разных моделей меняя гипер параметры:

### Дерево решений:

Напишим функцию которая будет менять глубину дерева от 1  до 10 и выберет лучшую модель дерева решений на основе Accuarry:

In [9]:
model_tree = None
best_result_1 = 0

for depth in range(1, 11):
    model_1 = DecisionTreeClassifier(random_state=1879, max_depth=depth)  
    model_1.fit(features_train,target_train) 
    result_1 = model_1.score(features_valid,target_valid) 
    if result_1 > best_result_1:
        model_tree = model_1
        best_result_1 = result_1 

Напечатаем результат:

In [10]:
print("Accuracy наилучшей модели на валидационной выборке:", best_result_1)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


Необходимая точность получена.

#### 2.Логистическая регрессия:

Инициализируем модель:

In [11]:
model_log = LogisticRegression(random_state=1879, solver='lbfgs') 

Обучим модель на тренировочной выборке:

In [12]:
model_log.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1879, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Проверим точность на валидациооной выборке и напечатаем результат:

In [13]:
result_2 = model_log.score(features_valid, target_valid)
print("Accuracy  модели логистической регрессии на валидационной выборке:", result_2)

Accuracy  модели логистической регрессии на валидационной выборке: 0.7293934681181959


Необходимая точность не получена.

### Cлучайный лес:

Напишим функцию которая будет менять число деревьевот 1 до 50 и выберет лучшую модель на основе Accuarry:

In [14]:
model_forest = None
best_result = 0
for est in range(1, 51):
    model = RandomForestClassifier(random_state=1879, n_estimators=est) 
    model.fit(features_train,target_train) 
    result = model.score(features_valid,target_valid) 
    if result > best_result:
        model_forest = model
        best_result = result 


Напечатаем результат:

In [15]:
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314


Необходимая точность получена.

Необходимая точность получена у двух моделей:

In [16]:
model_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=1879,
                       verbose=0, warm_start=False)

In [17]:
model_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1879, splitter='best')

In [18]:
start_time = time.time()
model_forest.predict(features_valid)
print("Время выполнения решения model_forest %s секунд" % (time.time() - start_time))

Время выполнения решения model_forest 0.014688491821289062 секунд


In [19]:
start_time = time.time()
model_tree.predict(features_valid)
print("Время выполнения решения model_tree %s секунд" % (time.time() - start_time))

Время выполнения решения model_tree 0.0016391277313232422 секунд


### Выводы

Точность модели дерева решений  0.78, а точность модели случайного леса 0.79. При этом скорость выполнения у модели случайного леса выше чем у дерева решений. Основываясь на том что нам нужна точность не ниже 0.75 считаю что оптимальной для данной задачи будет модель model_tree

## Проверка качества модели на тестовой выборке

### Проверим качество модели на тестовой выборке:
Пололучим предсказания по тестовой выборке:

In [20]:
predictions_test = model_tree.predict(features_test)

Получим предсказания по обучающей выборке:

In [21]:
predictions_train = model_tree.predict(features_train)

Получим точность модели на обучающей выборке:

In [22]:
accuarry_train = accuracy_score(target_train, predictions_train)

Получим точность модели на тестовой выборке:

In [23]:
accuarry_test = accuracy_score(target_test, predictions_test)

Напечатаем результаты:

In [24]:
print('Accuracy')
print('Обучающая выборка:', accuarry_train)
print('Тестовая выборка:', accuarry_test)

Accuracy
Обучающая выборка: 0.8475103734439834
Тестовая выборка: 0.8040435458786936


Судя по результатом модель получилось слегка переученой, но необходимую точность на тестовой выборке мы получили.

### Проверка модели на вменяемость 

Посмотрим какой самый часто встречающейся класс в тестовой выборке:

In [25]:
target_test.value_counts()

0    463
1    180
Name: is_ultra, dtype: int64

Передадим моделе данные состоящие только из 0

Создаем список,нулей нужного размера:

In [26]:
predictions_adequacy = [0]*target_test.count()

Делаем проверку точности:

In [27]:
adequacy_test = accuracy_score(target_test, predictions_adequacy)

Напечатаем результаты:

In [28]:
print('Accuracy')
print('Accurac при одинаковых ответах:', adequacy_test)
print('Тестовая выборка:', accuarry_test)

Accuracy
Accurac при одинаковых ответах: 0.7200622083981337
Тестовая выборка: 0.8040435458786936


Судя по результатом выше, модель действительно обучилась.

## Выводы

Необходимая для решения поставленой задачи модель выбрана, она удовлетворяет требованию - доля правильных ответов должна быть не ниже 0.75, на тестовой выборке модель показала точность 0.8 , и имеет оптимальную скорость решения. Модель проверена на вменяемость - она действительно обучилась.
